In [ ]:
""" 下载数据集 """
import dataHandler

dataHandler.download_data_from_baostock(start_date="2025-04-04", end_date="2025-05-19", adjustflag = "1")
dataHandler.transform_csv_into_bin("2025-04-07~2025-05-19")

e:\PythonProject\rich


2025-05-20 13:20:28.835 | INFO     | dump_bin:_get_all_date:273 - start get all date......
100%|██████████| 4935/4935 [00:27<00:00, 176.82it/s]
2025-05-20 13:20:56.750 | INFO     | dump_bin:_get_all_date:292 - end of get all date.

2025-05-20 13:20:56.752 | INFO     | dump_bin:_dump_calendars:295 - start dump calendars......
2025-05-20 13:20:56.756 | INFO     | dump_bin:_dump_calendars:298 - end of calendars dump.

2025-05-20 13:20:56.756 | INFO     | dump_bin:_dump_instruments:301 - start dump instruments......
2025-05-20 13:20:56.766 | INFO     | dump_bin:_dump_instruments:303 - end of instruments dump.

2025-05-20 13:20:56.766 | INFO     | dump_bin:_dump_features:306 - start dump features......
100%|██████████| 4935/4935 [00:32<00:00, 152.89it/s]
2025-05-20 13:21:29.047 | INFO     | dump_bin:_dump_features:313 - end of features dump.



In [ ]:
""" 生成特征 """
import qlib
from qlib import data
from qlib.utils import init_instance_by_config

# 数据集初始化
qlib.init(provider_uri="E:/PythonProject/test/stock_data/qilb_cn_data_from_baostock/2024-04-10~2025-04-10")
stock_list = data.D.instruments(market = "all")

""" 参数配置 """
# 数据处理器参数配置：整体数据开始结束时间，训练集开始结束时间，股票池
data_handler_config = {
    "start_time": "2024-04-10",
    "end_time": "2025-04-10",
    "fit_start_time": "2024-04-10",
    "fit_end_time": "2024-09-23",
    "instruments": stock_list
}

# 任务参数配置
task = {
    # 机器学习模型参数配置
    "model": {
        "class": "LGBModel", # 模型类
        "module_path": "qlib.contrib.model.gbdt", # 模型类所在的模块
        "kwargs": { # 模型类超参数配置，使用这些参数初始化上面使用的模型类对象
            "loss": "mse",
            "colsample_bytree": 0.8879,
            "learning_rate": 0.0421,
            "subsample": 0.8789,
            "lambda_l1": 205.6999,
            "lambda_l2": 580.9768,
            "max_depth": 8,
            "num_leaves": 210,
            "num_threads": 20,
        },
    },
    # 因子数据参数配置
    "dataset": {
        "class": "DatasetH", # 数据集类，是Dataset with Data(H)andler的缩写，即带数据处理器的数据集
        "module_path": "qlib.data.dataset", # 数据集类所在的模块
        "kwargs": { # 数据集类的参数配置
            "handler": { # 数据集使用的数据处理器配置
                "class": "Alpha158", # 数据处理器类，继承自DataHandlerLP，特征
                "module_path": "qlib.contrib.data.handler", # 数据处理器类所在模块
                "kwargs": data_handler_config, # 数据处理器参数配置
            },
            "segments": { # 数据集时段划分
                "train": ("2024-04-10", "2024-09-23"), # 训练集时段
                "valid": ("2024-09-24", "2024-10-24"), # 验证集时段
                "test": ("2024-10-25", "2025-04-10"),  # 测试集时段
            },
        },
    },
}

# 实例化模型对象
model = init_instance_by_config(task["model"])
dataset = init_instance_by_config(task["dataset"])